# FastWOE Styled Display Demo

This notebook demonstrates the styled DataFrame rendering capabilities of FastWOE for Jupyter notebooks.

Features:

- **Interactive tables with itables** (if installed): Sorting, filtering, search, and pagination
- **Pandas Styler fallback**: Sticky column headers and gradient color-coding
- **Decorator-based styling**: Clean, reusable code patterns


In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from fastwoe import FastWoe
from fastwoe.display import iv_styled, styled

## 1. Generate Sample Data


In [2]:
# Generate synthetic credit risk data
np.random.seed(42)

prevalence = 0.05

X, y = make_classification(
    n_samples=5000,
    n_features=10,
    n_informative=6,
    n_redundant=2,
    n_classes=2,
    weights=[1 - prevalence, prevalence],  # Imbalanced like real credit data
    random_state=42,
)

# Create DataFrame with meaningful feature names
feature_names = [
    "income",
    "age",
    "employment_length",
    "debt_to_income",
    "credit_history",
    "loan_amount",
    "num_accounts",
    "utilization_rate",
    "payment_history",
    "inquiries",
]

X_df = pd.DataFrame(X, columns=feature_names)
y_series = pd.Series(y, name="default")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_series, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Default rate: {y.mean():.2%}")

Training samples: 3500
Test samples: 1500
Default rate: 5.52%


## 2. Fit FastWOE Model


In [3]:
# Initialize and fit WOE encoder
woe = FastWoe()
woe.fit(X_train, y_train)

# Transform data
X_train_woe = woe.transform(X_train)
X_test_woe = woe.transform(X_test)

print("✓ FastWOE model fitted successfully!")

✓ FastWOE model fitted successfully!


## 3. Information Value Analysis with @iv_styled Decorator

The `@iv_styled` decorator automatically applies IV analysis styling. If `itables` is installed, you'll get an interactive table with sorting, filtering, and search capabilities:


In [4]:
@iv_styled
def get_iv_analysis():
    """Get IV analysis with automatic styling."""
    return woe.get_iv_analysis()


# Automatically returns styled output
get_iv_analysis()

feature,iv,iv_se,iv_ci_lower,iv_ci_upper,iv_significance,n_categories,gini
inquiries,1.8697,0.2454,1.3888,2.3507,Significant,8,0.5942
income,1.6458,0.2527,1.1505,2.1411,Significant,8,0.5293
num_accounts,0.9174,0.1618,0.6003,1.2345,Significant,8,0.4244
age,0.7046,0.1361,0.4378,0.9713,Significant,8,0.3639
loan_amount,0.5733,0.1265,0.3254,0.8212,Significant,7,0.3545
debt_to_income,0.3490,0.1453,0.0642,0.6337,Significant,6,0.2265
utilization_rate,0.2998,0.1360,0.0334,0.5663,Significant,7,0.1495
credit_history,0.1999,0.1079,0.0000,0.4113,Not Significant,5,0.1280
employment_length,0.1551,0.0827,0.0000,0.3171,Not Significant,6,0.1084
payment_history,0.1374,0.0839,0.0000,0.3018,Not Significant,7,0.0933


## 4. Top Features with @styled Decorator

Use `@styled` decorator for custom styling:


In [5]:
# Light theme version
@styled(highlight_cols=["iv", "gini"], precision=4, theme="light")
def get_top_features_light(n=5):
    """Get top N features by IV (light theme)."""
    iv_df = woe.get_iv_analysis()
    return iv_df.nlargest(n, "iv")[["feature", "iv", "gini", "iv_se"]]


# Display both versions
print("### Light Theme")
get_top_features_light(n=5)

### Light Theme


feature,iv,gini,iv_se
inquiries,1.8697,0.5942,0.2454
income,1.6458,0.5293,0.2527
num_accounts,0.9174,0.4244,0.1618
age,0.7046,0.3639,0.1361
loan_amount,0.5733,0.3545,0.1265


In [6]:
# Dark theme version
@styled(highlight_cols=["iv", "gini"], precision=4, theme="dark")
def get_top_features_dark(n=5):
    """Get top N features by IV (dark theme)."""
    iv_df = woe.get_iv_analysis()
    return iv_df.nlargest(n, "iv")[["feature", "iv", "gini", "iv_se"]]


print("\n### Dark Theme")
get_top_features_dark(n=5)


### Dark Theme


feature,iv,gini,iv_se
inquiries,1.8697,0.5942,0.2454
income,1.6458,0.5293,0.2527
num_accounts,0.9174,0.4244,0.1618
age,0.7046,0.3639,0.1361
loan_amount,0.5733,0.3545,0.1265


## 5. WOE Mappings for Top Features


In [7]:
# Get top 3 features
top_features = woe.get_iv_analysis().nlargest(3, "iv")["feature"].tolist()

for feature in top_features:

    @styled(
        highlight_cols=["woe", "iv"],
        precision=4,
    )
    def show_mapping(feat=feature):  # ← Bind loop variable as default argument
        mappings = woe.get_all_mappings()[feat]
        return pd.DataFrame(mappings)

    display(show_mapping())

category,count,count_pct,good_count,bad_count,event_rate,woe,woe_se,woe_ci_lower,woe_ci_upper
"(-0.0, -0.0]",2,0.0571,0,2,1.0000,15.1039,1.0000,13.1439,17.0639
"(-0.0, 1.2]",657,18.7714,594,63,0.0959,0.5974,0.1325,0.3377,0.8571
"(-2.0, -0.0]",1740,49.7143,1696,44,0.0253,-0.8107,0.1527,-1.1100,-0.5114
"(-∞, -2.0]",802,22.9143,796,6,0.0075,-2.0467,0.4098,-2.8499,-1.2436
"(1.2, 1.2]",6,0.1714,2,4,0.6667,3.5342,0.8660,1.8369,5.2316
"(1.2, 2.5]",243,6.9429,195,48,0.1975,1.4393,0.1611,1.1235,1.7551
"(2.5, 3.9]",46,1.3143,24,22,0.4783,2.7541,0.2952,2.1756,3.3326
"(3.9, ∞)",4,0.1143,0,4,1.0000,15.7970,1.1547,13.5339,18.0602


category,count,count_pct,good_count,bad_count,event_rate,woe,woe_se,woe_ci_lower,woe_ci_upper
"(-1.1, -1.1]",2,0.0571,0,2,1.0000,15.1039,1.0000,13.1439,17.0639
"(-1.1, 0.1]",474,13.5429,438,36,0.0759,0.3424,0.1734,0.0026,0.6822
"(-3.6, -1.1]",348,9.9429,283,65,0.1868,1.3700,0.1375,1.1005,1.6396
"(-3.6, -3.6]",4,0.1143,0,4,1.0000,15.7970,1.1547,13.5339,18.0602
"(-∞, -3.6]",35,1.0000,18,17,0.4857,2.7839,0.3382,2.1211,3.4468
"(0.1, 0.1]",2,0.0571,0,2,1.0000,15.1039,1.0000,13.1439,17.0639
"(0.1, 1.5]",863,24.6571,830,33,0.0382,-0.3838,0.1775,-0.7317,-0.0359
"(1.5, ∞)",1772,50.6286,1738,34,0.0192,-1.0930,0.1732,-1.4324,-0.7536


category,count,count_pct,good_count,bad_count,event_rate,woe,woe_se,woe_ci_lower,woe_ci_upper
"(-0.8, -0.8]",1,0.0286,0,1,1.0000,14.4108,1.0000,12.4508,16.3707
"(-0.8, 0.0]",260,7.4286,239,21,0.0808,0.4092,0.2276,-0.0369,0.8553
"(-4.0, -0.8]",1736,49.6000,1667,69,0.0397,-0.3436,0.1229,-0.5844,-0.1028
"(-∞, -4.0]",1017,29.0571,997,20,0.0197,-1.0679,0.2258,-1.5105,-0.6253
"(0.0, 0.1]",8,0.2286,4,4,0.5000,2.8411,0.7071,1.4552,4.2270
"(0.1, 6.1]",467,13.3429,396,71,0.1520,1.1224,0.1289,0.8698,1.3750
"(6.1, 6.2]",2,0.0571,0,2,1.0000,15.1039,1.0000,13.1439,17.0639
"(6.2, ∞)",9,0.2571,4,5,0.5556,3.0642,0.6708,1.7495,4.3790


## 6. Feature Importance Ranking


In [8]:
@styled(
    highlight_cols=["Abs_Coefficient", "IV"],
    precision=4,
)
def get_feature_importance():
    """Calculate feature importance from model coefficients."""
    # Train model
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_woe, y_train)

    # Get IV analysis
    iv_df = woe.get_iv_analysis()

    return pd.DataFrame(
        {
            "Feature": feature_names,
            "Coefficient": model.coef_[0],
            "Abs_Coefficient": np.abs(model.coef_[0]),
            "IV": [iv_df[iv_df["feature"] == f]["iv"].values[0] for f in feature_names],
        }
    ).sort_values("Abs_Coefficient", ascending=False)


get_feature_importance()

Feature,Coefficient,Abs_Coefficient,IV
debt_to_income,-0.2121,0.2121,0.3490
age,0.0695,0.0695,0.7046
num_accounts,0.0592,0.0592,0.9174
inquiries,0.0552,0.0552,1.8697
utilization_rate,-0.0518,0.0518,0.2998
payment_history,-0.0482,0.0482,0.1374
loan_amount,0.0227,0.0227,0.5733
income,0.0188,0.0188,1.6458
credit_history,-0.0058,0.0058,0.1999
employment_length,0.0030,0.0030,0.1551


## 7. Risk Segmentation Analysis


In [9]:
@styled(
    highlight_cols=["Default Rate %", "Expected Loss"],
    precision=2,
)
def get_risk_segments():
    """Create risk segmentation table."""
    return pd.DataFrame(
        {
            "Segment": ["Very Low", "Low", "Medium", "High", "Very High"],
            "Score Range": ["0-200", "200-400", "400-600", "600-800", "800-1000"],
            "Population %": [0.15, 0.25, 0.30, 0.20, 0.10],
            "Default Rate %": [0.02, 0.05, 0.12, 0.25, 0.45],
            "Avg Exposure": [5000, 8000, 12000, 15000, 10000],
            "Expected Loss": [100, 400, 1440, 3750, 4500],
        }
    )


get_risk_segments()

Segment,Score Range,Population %,Default Rate %,Avg Exposure,Expected Loss
Very Low,0-200,0.15,0.02,5000,100
Low,200-400,0.25,0.05,8000,400
Medium,400-600,0.30,0.12,12000,1440
High,600-800,0.20,0.25,15000,3750
Very High,800-1000,0.10,0.45,10000,4500


## 8. Binning Summary with Decorator

Example showing WOE binning statistics:


In [10]:
@styled(
    highlight_cols=["woe", "iv"],
    precision=4,
)
def get_binning_summary():
    """Get binning summary for the top feature."""
    # Get top feature by IV
    top_feature = woe.get_iv_analysis().nlargest(1, "iv")["feature"].values[0]

    # Get binning summary
    summary = woe.get_binning_summary()
    return summary[summary["feature"] == top_feature]


get_binning_summary()

feature,values,n_bins,missing,method,monotonic_constraint
inquiries,3500,8,0,tree,0
